In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [39]:
import streamlit as st

In [40]:
# from google.colab import drive
# drive.mount('/content/drive')

In [41]:
data_path = 'carbike-class\dataset\Car-Bike-Dataset'

In [42]:
!pip install scikit-learn

In [43]:
import sklearn as sk

In [86]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix


In [45]:
data_dir = "dataset/Car-Bike-Dataset"
input_size= (128, 128) # Các ảnh sẽ được resize về một kích thước chung, độ phân giải càng cao càng tốn tài nguyên (CPU, GPU, RAM,..)
val_frac = 0.1         # Tỉ lệ của tập validation sẽ được chia ra từ tập train, thường chia theo các tỉ lệ 9:1, 8:2, 7:3
batch_size = 128       # Số ảnh xử lý/lần để cập nhật tham số, số càng cao càng tốn tài nguyên

# Tiền xử lý dữ liệu ảnh để mô hình học tốt hơn
data_augmentor = ImageDataGenerator(samplewise_center=True,            # Chuẩn hoá mean của các giá trị pixel
                                    samplewise_std_normalization=True, # Chuẩn hoá khoảng giá trị của các pixel
                                    validation_split=val_frac)

# Đọc dữ liệu từ thư mục và tiền xử lý
train_generator = data_augmentor.flow_from_directory(data_dir,
                                                     target_size=input_size,
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     subset="training")
val_generator = data_augmentor.flow_from_directory(data_dir,
                                                   target_size=input_size,
                                                   batch_size=batch_size,
                                                   subset="validation")

Found 3600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [46]:
train_generator.class_indices
len(train_generator.class_indices)

2

In [47]:
num_classes = len(train_generator.class_indices) # 29
input_shape = (128, 128, 3)                      # 3 chiều cho ảnh RGB

# Cấu trúc mô hình tham khảo
model = keras.models.Sequential([
    keras.Input(shape=input_shape),
    # block 1
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),     # Layer tích chập
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),     # Thêm Layer tích chập để học nhiều chi tiết hơn
    keras.layers.MaxPooling2D(pool_size=(2, 2)),                        # Layer MaxPool luôn theo sau layer Conv
    keras.layers.Dropout(0.5),                                          # Layer Dropout để mô hình không bị thiên kiến
    # block 2
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),     # Cấu trúc giống với block 1, tăng chiều từ 32 lên 64
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.5),
    # block 3
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),    # Cấu trúc giống với block 1, tăng chiều từ 64 lên 128
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.5),
    # FCN block chịu trách nhiệm phân loại
    keras.layers.Flatten(),                                            # Chỉ có một layer flatten trong mô hình
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation='sigmoid'),             # Layer Dense cuối có chiều là tổng số class
])
model.compile(optimizer='adam',                 # Luôn sử dụng `adam`
              loss='binary_crossentropy',  # `binary_crossentropy` cho lớp True/False, `categorical_crossentropy` cho nhiều lớp
              metrics=["accuracy"])             # Đánh giá mô hình dựa trên độ chính xác
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 124, 124, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 60, 60, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 58, 58, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 27, 27, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 25, 25, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │     9,437,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,790,626 (37.35 MB)

 Trainable params: 9,790,626 (37.35 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator, 
)

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.5275 - loss: 0.7579 - val_accuracy: 0.5625 - val_loss: 0.6858
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.6250 - loss: 0.6603 - val_accuracy: 0.7375 - val_loss: 0.5921
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.7566 - loss: 0.5260 - val_accuracy: 0.7950 - val_loss: 0.4461
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.8562 - loss: 0.3610 - val_accuracy: 0.8050 - val_loss: 0.4280
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.8642 - loss: 0.3211 - val_accuracy: 0.8800 - val_loss: 0.2691
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.9011 - loss: 0.2247 - val_accuracy: 0.9125 - val_loss: 0.2227
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.9314 - loss: 0.1846 - val_accuracy: 0.9225 - val_loss: 0.2007
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.9274 - loss: 0.1739 - val_accuracy: 0.9000 - val_loss:

In [67]:
model.save('carbike-class2\model3.keras')

In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [85]:
checkpoint = ModelCheckpoint(
    filepath='carbike-class2\model5.keras',  # Tên mẫu của file (v.d. checkpoint tại epoch 1 là `model_epoch_01.keras`)
    save_weights_only=False,                   # Lưu cả cấu trúc và thông số của mô hình
    save_best_only=False,                      # Set về True để chỉ lưu khi mô hình cải thiện
    monitor='val_accuracy',                        # Chọn thang đo để theo dõi mô hình (v.d. 'val_loss', 'val_accuracy')
    verbose=1                                  # Hiển thị thông tin ra màn hình khi mô hình được checkpoint
)

# Huấn luyện mô hình với config đã chọn
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=4,                            # Số lần train toàn bộ dữ liệu
    callbacks=[checkpoint]               # Thêm config checkpoint
)

Epoch 1/4
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 939ms/step - accuracy: 0.9541 - loss: 0.1141
Epoch 1: saving model to carbike-class2\model5.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 994ms/step - accuracy: 0.9540 - loss: 0.1144 - val_accuracy: 0.9275 - val_loss: 0.1741
Epoch 2/4
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 971ms/step - accuracy: 0.9629 - loss: 0.0943
Epoch 2: saving model to carbike-class2\model5.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.9628 - loss: 0.0946 - val_accuracy: 0.9025 - val_loss: 0.2509
Epoch 3/4
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 976ms/step - accuracy: 0.9650 - loss: 0.1000
Epoch 3: saving model to carbike-class2\model5.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.9650 - loss: 0.0998 - val_accuracy: 0.9250 - val_loss: 0.1643
Epoch 4/4
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 951ms/step - accuracy: 0.9698 - loss: 0.0699
Epoch 4: saving model to carbike-class2\model5.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 996ms/step - accuracy: 0.9697 - loss: 0.0702 - val_accuracy: 0.9325 -

In [99]:
import numpy as np
from tensorflow.keras.saving import load_model
import os

# Use the correct path as used in ModelCheckpoint and model.save
model_path = r'carbike-class2\model5.keras'
if not os.path.exists(model_path):
	raise FileNotFoundError(f"File not found: {model_path}. Please check the path and filename.")

loaded_model = load_model(model_path)

In [100]:
%pip install matplotlib opencv-python

import matplotlib.pyplot as plt
import cv2
from glob import glob

Note: you may need to restart the kernel to use updated packages.


In [101]:
classes = train_generator.class_indices
classes = dict((v,k) for k,v in classes.items())
classes

{0: 'Bike', 1: 'Car'}

In [152]:
test_image = 'Test-Dir\Test-imgs\Bike\2017-Bajaj-Pulsar-150-Specifications.jpg'
print(test_image)


Test-Dir\Test-imgs\Bike7-Bajaj-Pulsar-150-Specifications.jpg


In [156]:
# Display the test image using the test_images path

img_path = test_image
im = cv2.imread(img_path)
if im is None:
    print(f"Failed to load image at {img_path}. Please check the path or filename.")
else:
    plt.rcParams['figure.figsize'] = (2.0, 2.0)
    plt.title(os.path.basename(img_path))
    plt.axis('off')
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    plt.show()


Failed to load image at Test-Dir\Test-imgs\Bike7-Bajaj-Pulsar-150-Specifications.jpg. Please check the path or filename.


In [157]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
classes = train_generator.class_indices


classes = {v: k for k, v in classes.items()}


# Use the available image_path variable to avoid FileNotFoundError
img = image.load_img(image_path, target_size=(128, 128))  # Resize to (128, 128)
img_array = image.img_to_array(img)  # Convert from image to numpy array
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension: batch_size=1

# Create data generator with preprocessing
test_datagen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True
)
img_generator = test_datagen.flow(img_array, batch_size=1)  # Use `flow`

# Make prediction
prediction = loaded_model.predict(next(img_generator))  # Predict using the model
prediction_label = np.argmax(prediction)  # Get the predicted label

# Check if the predicted label exists in `classes`
if prediction_label in classes:
    print("Prediction:", prediction_label)  # Print the class name
else:
    print("Prediction label not found in classes. Please check your model's output.")


FileNotFoundError: [Errno 2] No such file or directory: 'carbike-class\\Test-imgs\\Car\\1552676931-8228.jpg'

In [163]:
# Use the correct test directory path as in your dataset
test_dir = r'Test-Dir/Test-imgs'
print("Absolute test_dir path:", os.path.abspath(test_dir))
if not os.path.exists(test_dir):
    raise FileNotFoundError(f"Directory does not exist: {test_dir}")

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_size,
    batch_size=1,
    shuffle=False,
    class_mode=None
)


Absolute test_dir path: c:\Users\Quân\Desktop\study\carbike-class\Test-Dir\Test-imgs


FileNotFoundError: Directory does not exist: Test-Dir/Test-imgs

In [165]:
preds = loaded_model.predict(test_generator, verbose=1)
preds = np.argmax(preds, axis=1)
preds_labels = [classes[pred] for pred in preds]
print(preds_labels)

NameError: name 'test_generator' is not defined

In [166]:
val_loss, val_acc = loaded_model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - accuracy: 0.9290 - loss: 0.1718
Validation Accuracy: 93.25%
